# BiLSTM training

In [1]:
import pandas as pd
import numpy as np
import os
import json
import sagemaker

from sagemaker.tensorflow import TensorFlow

In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

print(f'Bucket: {bucket}')
print(f'Role: {role}')

Bucket: sagemaker-eu-west-1-087816224558
Role: arn:aws:iam::087816224558:role/service-role/AmazonSageMaker-ExecutionRole-20200424T125478


## Load constants and directories

In [3]:
constants = json.load( open( "utils/objects/constants_dict.json", "r" ) )
tag2idx = json.load( open("utils/objects/tag2idx.json", "r"))
data_directories = json.load( open("utils/objects/data_directories.json", "r"))

In [4]:
print('Constants:')
display(constants)
print('Directories:')
display(data_directories)

Constants:


{'N_WORDS': 35178, 'N_TAGS': 17, 'MAX_LEN': 45}

Directories:


{'train_data_directory': 's3://sagemaker-eu-west-1-087816224558/named_entity_recognition/bilstm_data/bilstm_train.csv',
 'test_data_directory': 's3://sagemaker-eu-west-1-087816224558/named_entity_recognition/bilstm_data/bilstm_test.csv'}

## Training script

Uncomment cell below to display training script from source_bilstm

In [ ]:
# ! pygmentize models/train_bilstm.py

### Hyperparameters

Number of epochs is defined to be higher that it possible become - training stops after validation loss stops decreasing or reaches target state (check training script) to prevent overfitting. - it turned out to be 7

Conclusions from tuning:
- Smaller batch size (16) results in four times higher loss, same with higher (64) - finally I choosed 32.  
- Higher (than 256) hidden unit size results in overfitting, 256 seems to be optimal.
- Best embedding has been achieved by specifing its dimension to 128.


#### Final hyperparameters set:

In [5]:
hyperparameters={
    'epochs': 20,
    'batch-size': 32,
    'embed-dim': 128,
    'hidden-units': 256,
    'max-len': constants['MAX_LEN'],
    'n-tags': constants['N_TAGS'],
    'n-words': constants['N_WORDS'],
    'model-version': '1'
}

### Declare estimator and train the model

In [8]:
tf_estimator = TensorFlow(entry_point='train_bilstm.py', 
                          source_dir="source_bilstm",
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p2.xlarge',
                          framework_version='2.1.0', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters=hyperparameters
                         )

In [9]:
tf_estimator.fit({'training': data_directories['train_data_directory']})

2020-06-05 13:02:21 Starting - Starting the training job...
2020-06-05 13:02:23 Starting - Launching requested ML instances......
2020-06-05 13:03:27 Starting - Preparing the instances for training.........
2020-06-05 13:05:05 Downloading - Downloading input data......
2020-06-05 13:05:54 Training - Downloading the training image..
2020-06-05 13:06:27 Training - Training image download completed. Training in progress.2020-06-05 13:06:32,751 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-06-05 13:06:33,227 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "embed-dim": 128,
        "batch-size": 32,
        "max-len": 45,
    

## Deploy model

In [10]:
tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge',
                                   endpoint_name='ner-bilstm-v8')

-------------!

In [11]:
tf_predictor.endpoint

'ner-bilstm-v8'

## Clean up (optional)

In [ ]:
# tf_predictor.delete_endpoint()

In [ ]:
# import boto3
# bucket_to_delete = boto3.resource('s3').Bucket(bucket)
# bucket_to_delete.objects.all().delete()